In [1]:
import ccxt
print('CCXT Version:', ccxt.__version__)
exchange = ccxt.binance({
    'enableRateLimit': True,
    'apiKey': '6f451265b0590b2776adc8079c3875460c0b11177dd690f66a780f37c39a245f',
    'secret': '476acb93f59148c133c5ed93b33e456f0e38eb1e530b505d139e34c4eb9b3ea3',
    'options': {
        'defaultType': 'future',
    },
    'hedgeMode': True
})

exchange.set_sandbox_mode(True)

markets = exchange.load_markets()

exchange.verbose = True

balance = exchange.fetch_balance()

CCXT Version: 1.65.15

fetch Request: binance GET https://testnet.binancefuture.com/fapi/v2/account?timestamp=1640874745596&recvWindow=5000&signature=355385e550d4c3946565b5b1fd48641496f08b42a3234ac21651a217e217b8b1 RequestHeaders: {'X-MBX-APIKEY': '6f451265b0590b2776adc8079c3875460c0b11177dd690f66a780f37c39a245f', 'User-Agent': 'python-requests/2.26.0', 'Accept-Encoding': 'gzip, deflate'} RequestBody: None

fetch Response: binance GET https://testnet.binancefuture.com/fapi/v2/account?timestamp=1640874745596&recvWindow=5000&signature=355385e550d4c3946565b5b1fd48641496f08b42a3234ac21651a217e217b8b1 200 ResponseHeaders: {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Thu, 30 Dec 2021 14:32:26 GMT', 'Server': 'Bengine', 'Vary': 'Accept-Encoding', 'X-MBX-USED-WEIGHT-1M': '6', 'x-response-time': '7ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentia

In [ ]:
response = exchange.fapiPrivate_post_positionside_dual({'dualSidePosition': True, })

In [29]:
def buy(exchange, symbol, order_type, amount, side='buy'):
    order_params = {'positionSide':'LONG','recvWindow': 10000000}
    main_order = exchange.create_order(symbol, order_type, side, amount, None, order_params)
    return main_order

def sell(exchange, symbol, order_type, amount, side='sell'):
    order_params = {'positionSide':'SHORT','recvWindow': 10000000}
    main_order = exchange.create_order(symbol, order_type, side, amount, None, order_params)
    return main_order

In [30]:
def close_long(exchange, symbol, main_order):
    exchange.create_order(symbol, 'market', "sell", main_order['amount'], None, params={'positionSide':'LONG','recvWindow': 10000000})

def close_short(exchange, symbol, main_order):
    exchange.create_order(symbol, 'market', "buy", sell_main_order['amount'], None, params={'positionSide':'SHORT','recvWindow': 10000000})

In [40]:
import pandas as pd
trade_df = pd.DataFrame({"a":[2]})
trade_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\renko_long_signal.csv",index=False)
trade_df = pd.DataFrame({"a":[2]})
trade_df.to_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\renko_short_signal.csv",index=False)
trade_status = pd.DataFrame({"status":['Open']})
trade_status.to_csv('trade_status.csv', index = False)

In [ ]:
import time
import datetime
from IPython.display import display, clear_output
t = 0

main_order = None
sell_main_order = None
symbol = 'BTC/USDT'
position_symbol = 'BTCUSDT'

while True:
    clear_output(wait=True)
    try:
        long_ind_df = pd.read_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\renko_long_signal.csv", sep=";",names=["Ind"])
    except:
        continue
    try:
        short_ind_df = pd.read_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\renko_short_signal.csv", sep=";",names=["Ind"])
    except:
        continue
    try:
        long_close_df = pd.read_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\long_allow.csv", sep=";",names=["Ind"])
    except:
        continue
    long_closure = long_close_df.Ind.tolist()[0]
    try:
        short_close_df = pd.read_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\short_allow.csv", sep=";",names=["Ind"])
    except:
        continue
    short_closure = short_close_df.Ind.tolist()[0]
    long_signal = long_ind_df.Ind.tolist()[0]
    short_signal = short_ind_df.Ind.tolist()[0]
    status_df = pd.read_csv('trade_status.csv')
    status = status_df.status.tolist()[0]
    if long_signal == 0 and status == 'Open' and long_closure == 1:
        try:
            main_order = buy(exchange,symbol,'market',0.001)
        except:
            continue
        trade_status = pd.DataFrame({"status":['buy_on']})
        trade_status.to_csv('trade_status.csv', index = False)
    if short_signal == 1 and status == 'Open' and short_closure == 1:
        try:
            sell_main_order = sell(exchange,symbol,'market',0.001)
        except:
            continue
        trade_status = pd.DataFrame({"status":['sell_on']})
        trade_status.to_csv('trade_status.csv', index = False)
    try:
        long_close_df = pd.read_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\long_allow.csv", sep=";",names=["Ind"])
    except:
        continue
    long_closure = long_close_df.Ind.tolist()[0]
    if long_closure == 0 and main_order is not None:
        close_long(exchange, symbol, main_order)
        trade_status = pd.DataFrame({"status":['Open']})
        trade_status.to_csv('trade_status.csv', index = False)
        main_order = None
    try:
        short_close_df = pd.read_csv(r"C:\Users\91984\AppData\Roaming\MetaQuotes\Terminal\AE925E3426D43DD22F06664D804D4525\MQL4\Files\short_allow.csv", sep=";",names=["Ind"])
    except:
        continue
    short_closure = short_close_df.Ind.tolist()[0]
    if short_closure == 0 and sell_main_order is not None:
        close_short(exchange, symbol, sell_main_order)
        trade_status = pd.DataFrame({"status":['Open']})
        trade_status.to_csv('trade_status.csv', index = False)
        sell_main_order = None
    time.sleep(3)